In [ ]:
from transformers import AutoTokenizer, 

In [2]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("Unbabel/XCOMET-XL")

OSError: Unbabel/XCOMET-XL is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [1]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/XCOMET-XL")
model = load_from_checkpoint(model_path)
data = [
    {
        "src": "Boris Johnson teeters on edge of favour with Tory MPs", 
        "mt": "Boris Johnson ist bei Tory-Abgeordneten völlig in der Gunst", 
        "ref": "Boris Johnsons Beliebtheit bei Tory-MPs steht auf der Kippe"
    }
]
model_output = model.predict(data, batch_size=8, gpus=1)
# Segment-level scores
print (model_output.scores)

# System-level score
print (model_output.system_score)

# Score explanation (error spans)
print (model_output.metadata.error_spans)


/home/s3/hyeryung/miniconda3/envs/loc-edit/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


KeyError: "Model 'Unbabel/XCOMET-XL' not supported by COMET."

# Toxic Spans Detection Dataset

In [1]:
import os
from collections import defaultdict
os.chdir('/home/s3/hyeryung/mucoco')

from transformers import AutoTokenizer
import pandas as pd

In [282]:
def get_token2index(tmp_text, tmp_tokens, spans):
    j = 0
    token2index = defaultdict(list)
    tmp_token = tmp_tokens[j]
    for i in range(len(tmp_text)):
        # print(i, tmp_text[i], tmp_token)
        while True:
            # print(i, tmp_text[i], tmp_token)
            if tmp_text[i].replace(' ', 'Ġ').replace('\n', 'Ċ').replace('’', 'â') == tmp_token[0]:
                tmp_token = tmp_token[1:]
                token2index[j].append(i)
                if len(tmp_token) == 0:
                    j += 1
                    if j >= len(tmp_tokens):
                        break
                    tmp_token = tmp_tokens[j]
                break
            else:
                j += 1
                if j >= len(tmp_tokens):
                    break
                tmp_token = tmp_tokens[j]
        if j >= len(tmp_tokens):
            break
    index2token={tuple(v):k for k,v in token2index.items()}  

    tokens_ix = []
    for sp in spans:
        for tk, v in index2token.items():
            if sp in tk:
                tokens_ix.append(v)
                break
    
    return sorted(list(set(tokens_ix)))

In [2]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [279]:
data_path = "data/toxicity/toxic_spans/SemEval2021/data/tsd_test.csv"
data = pd.read_csv(data_path)

In [280]:
data['tokens']=data['text'].apply(lambda x: tokenizer.tokenize(x))

In [281]:
data['spans']=data['spans'].apply(eval)

In [283]:
data['spans_tokens']=data.apply(lambda x: get_token2index(x['text'], x['tokens'], x['spans']),axis=1)

In [285]:
data.to_json("data/toxicity/toxic_spans/SemEval2021/data/tsd_test_spans.jsonl",lines=True,orient="records")

In [286]:
data_path = "data/toxicity/toxic_spans/SemEval2021/data/tsd_train.csv"
data = pd.read_csv(data_path)

In [287]:
data['tokens']=data['text'].apply(lambda x: tokenizer.tokenize(x))

In [288]:
data['spans']=data['spans'].apply(eval)

In [289]:
data['spans_tokens']=data.apply(lambda x: get_token2index(x['text'], x['tokens'], x['spans']),axis=1)

In [290]:
data.to_json("data/toxicity/toxic_spans/SemEval2021/data/tsd_train_spans.jsonl",lines=True,orient="records")

In [291]:
data_path = "data/toxicity/toxic_spans/SemEval2021/data/tsd_trial.csv"
data = pd.read_csv(data_path)

In [292]:
data['tokens']=data['text'].apply(lambda x: tokenizer.tokenize(x))

In [293]:
data['spans']=data['spans'].apply(eval)

In [294]:
data['spans_tokens']=data.apply(lambda x: get_token2index(x['text'], x['tokens'], x['spans']),axis=1)

In [295]:
data.to_json("data/toxicity/toxic_spans/SemEval2021/data/tsd_trial_spans.jsonl",lines=True,orient="records")

# MQM Dataset

In [1]:
import os
from collections import defaultdict
os.chdir('/home/s3/hyeryung/mucoco')

from transformers import AutoTokenizer
import pandas as pd

In [13]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [18]:
tokenizer.add_tokens(['<v>', '</v>'])

2

In [2]:
data_path = 'data/MT/wmt-mqm-human-evaluation/newstest2020/ende/mqm_newstest2020_ende.tsv'

In [3]:
data = pd.read_csv(data_path, sep='\t')

In [30]:
data['target_cleaned']=data['target'].str.replace('<v>','').str.replace('</v>','')

In [ ]:
data['target_tokenized']=data['target'].apply(tokenizer.tokenize(x))

In [34]:
data['target_cleaned'][3]

'Ehemaliger Bodyguard berichtet: Michael Jackson trug Klebeband auf seiner Nase, um auf die Titelseiten zu kommen'

In [20]:
tokens = tokenizer.tokenize(data['target'][0])

In [22]:
tokens.index('<v>')

18

In [23]:
tokens.index('</v>')

25

In [25]:
tokens[18+1:25]

['Ġz', 'u', 'Ġbe', 'k', 'omm', 'en']

In [7]:
data['target'][1]

'Michael Jackson trug Klebeband auf der Nase, um Titelseiten zu <v>bekommen,</v> behauptet ehemaliger Bodyguard'

In [8]:
data['target'][2]

'Michael Jackson trug Klebeband auf der Nase, um <v>Titelseiten zu bekommen</v>, behauptet ehemaliger Bodyguard'

In [11]:
data['target'][]

,system,doc,doc_id,seg_id,rater,source,target,category,severity
0,eTranslation.737,independent.281139,1,1,rater2,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",Style/Awkward,Minor
1,eTranslation.737,independent.281139,1,1,rater4,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",Accuracy/Mistranslation,Major
2,eTranslation.737,independent.281139,1,1,rater6,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",Accuracy/Mistranslation,Minor
3,Human-A.0,independent.281139,1,1,rater2,Michael Jackson wore tape on his nose to get f...,Ehemaliger Bodyguard berichtet: Michael Jackso...,No-error,no-error
4,Human-A.0,independent.281139,1,1,rater4,Michael Jackson wore tape on his nose to get f...,Ehemaliger Bodyguard berichtet: Michael Jackso...,No-error,no-error
...,...,...,...,...,...,...,...,...,...
79015,Tencent_Translation.1520,en.ndtv.com.13041,14,1418,rater2,A bust of the social reformer was vandalised d...,Eine Büste des Sozialreformers wurde während e...,Accuracy/Untranslated text,Minor
79016,Tencent_Translation.1520,en.ndtv.com.13041,14,1418,rater4,A bust of the social reformer was vandalised d...,Eine Büste des Sozialreformers wurde während e...,Terminology/Inappropriate for context,Minor
79017,Tohoku-AIP-NTT.890,en.ndtv.com.13041,14,1418,rater1,A bust of the social reformer was vandalised d...,Eine Büste des Sozialreformers wurde während e...,Accuracy/Mistranslation,Minor
79018,Tohoku-AIP-NTT.890,en.ndtv.com.13041,14,1418,rater2,A bust of the social reformer was vandalised d...,Eine Büste des Sozialreformers wurde während e...,Accuracy/Untranslated text,Minor
